# Seahorse Normalization Pipeline

In [ ]:

# Install dependencies if needed
# !pip install -r requirements.txt


In [ ]:

import os
import zipfile
import numpy as np
import pandas as pd
import cv2
from skimage.io import imread, imsave
from skimage.exposure import rescale_intensity
from skimage.util import img_as_ubyte
from cellpose import denoise, io, utils
from imaris_ims_file_reader import ims


In [ ]:

# Input configuration
FOLDER_PATH = ''         # path to images or zip
UPLOAD_ZIP = False       # set True if providing a zip file
NORMALIZE = True         # normalize Seahorse Excel file
NUCLEAR_CHANNEL = 1      # index starting from 1
CROP_TO_CENTER = True


In [ ]:

# --- helper functions ---

def convert_ims_to_tiff(src, dst, channel):
    os.makedirs(dst, exist_ok=True)
    for fname in os.listdir(src):
        if fname.endswith('.ims'):
            path = os.path.join(src, fname)
            out = os.path.join(dst, f"{os.path.splitext(fname)[0]}.tiff")
            reader = ims(path)
            reader.save_multilayer_tiff_stack(location=out, time_point=0,
                channel=channel-1, resolution_level=0)
            reader.close()


def histogram_bounds(images):
    hists = []
    for p in images:
        img = imread(p)
        if img.dtype == np.uint16:
            h, _ = np.histogram(img, bins=65536, range=(0,65535))
            hists.append(h)
    hist = np.mean(hists, axis=0)
    maxc = hist.max()
    lower = np.argmax(hist >= maxc*0.30)
    upper = len(hist) - 1 - np.argmax(hist[::-1] >= maxc*0.05)
    return round(lower*1.05), min(round(upper*1.3), 65535)


def crop_image(img, center=True):
    img8 = (img/256).astype(np.uint8)
    _, binary = cv2.threshold(img8,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    blurred = cv2.GaussianBlur(binary,(0,0),4)
    _, binary = cv2.threshold(blurred,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    dilated = cv2.dilate(binary,np.ones((3,3),np.uint8),iterations=200)
    n,_,stats,_ = cv2.connectedComponentsWithStats(dilated)
    stats = [s for s in stats[1:] if s[cv2.CC_STAT_AREA] > 10_000_000]
    if not stats:
        return img
    x,y,w,h = stats[0][:4]
    crop = img[y:y+h, x:x+w]
    if center:
        cy,cx = crop.shape[0]//2, crop.shape[1]//2
        mask = np.zeros_like(crop, dtype=np.uint8)
        cv2.circle(mask, (cx,cy), 1500, 255, -1)
        crop = cv2.bitwise_and(crop, crop, mask=mask)
        yv,xv = np.where(mask)
        crop = crop[yv.min():yv.max(), xv.min():xv.max()]
    else:
        crop = crop[100:-100,100:-100]
    return crop


def process_images(input_dir, nuclear_channel, crop_to_center):
    tiff_dir = os.path.join(input_dir, 'tiff_conversion')
    out_dir = os.path.join(input_dir, 'output')
    crop_dir = os.path.join(out_dir, 'Cropped')
    os.makedirs(crop_dir, exist_ok=True)
    convert_ims_to_tiff(input_dir, tiff_dir, nuclear_channel)
    images = [os.path.join(tiff_dir,f) for f in os.listdir(tiff_dir) if f.endswith('.tiff')]
    low, high = histogram_bounds(images)
    for p in images:
        img = imread(p)
        img = rescale_intensity(img, in_range=(low, high), out_range=(0,65535)).astype(np.uint16)
        cropped = crop_image(img, crop_to_center)
        imsave(os.path.join(crop_dir, os.path.basename(p)), cropped)
    return crop_dir


In [ ]:

# --- segmentation and normalization ---

def load_images(folder):
    return [io.imread(os.path.join(folder,f)) for f in os.listdir(folder) if f.endswith('.tiff')]


def segment_images(imgs, model_type, restore, channel, flow_th, cellprob_th):
    model = denoise.CellposeDenoiseModel(gpu=True, model_type=model_type,
                                         restore_type=f'{restore}_{model_type}')
    masks = []
    for img in imgs:
        if len(img.shape)==3 and img.shape[0]==min(img.shape):
            img = img.transpose(1,2,0)
        m,_,_,_ = model.eval(img, channels=[channel-1,0],
                             flow_threshold=flow_th,
                             cellprob_threshold=cellprob_th)
        masks.append(m.astype(np.uint16))
    return masks


def count_cells(masks, crop_to_center):
    counts = []
    for m in masks:
        if crop_to_center:
            h,w = m.shape
            cy,cx = h//2,w//2
            radius = int(0.98*min(h,w)/2)
            rr,cc = utils.disk((cy,cx), radius, shape=m.shape)
            roi = np.zeros_like(m, bool)
            roi[rr,cc]=True
            m = np.where(roi, m, 0)
        labels = np.unique(m)
        counts.append(len(labels[labels!=0]))
    return counts


def normalize_excel(excel_path, counts, wells, out_dir, column='OCR'):
    rate = pd.read_excel(excel_path, sheet_name='Rate')
    rate = rate[~rate['Group'].isin(['Blank','Background'])]
    counts = pd.Series(counts, index=wells)
    data = rate[['Time','Measurement','Well', column]].copy()
    def norm(row):
        if row['Well'] in counts:
            row[column] /= counts[row['Well']]/1000
        return row
    data = data.apply(norm, axis=1)
    out = os.path.join(out_dir, 'Processed_Seahorse_Data.xlsx')
    data.pivot(index='Time', columns='Well', values=column).to_excel(out)
    return out


In [ ]:

# Example execution
if UPLOAD_ZIP and FOLDER_PATH.endswith('.zip'):
    with zipfile.ZipFile(FOLDER_PATH,'r') as zf:
        extract_dir = os.path.join(os.path.dirname(FOLDER_PATH),'Extracted_Files')
        os.makedirs(extract_dir, exist_ok=True)
        zf.extractall(extract_dir)
        FOLDER_PATH = extract_dir

cropped = process_images(FOLDER_PATH, NUCLEAR_CHANNEL, CROP_TO_CENTER)
images = load_images(cropped)
mask_list = segment_images(images, 'cyto3', 'oneclick', NUCLEAR_CHANNEL, 0.4, 0)
counts = count_cells(mask_list, CROP_TO_CENTER)

if NORMALIZE:
    excel = [f for f in os.listdir(FOLDER_PATH) if f.endswith(('.xlsx','.xls'))]
    if excel:
        out = normalize_excel(os.path.join(FOLDER_PATH,excel[0]), counts,
                               [os.path.splitext(os.path.basename(f))[0] for f in os.listdir(cropped)],
                               cropped)
        print('Data saved to', out)
